In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import folium
import warnings
import json
warnings.simplefilter(action = "ignore", category = FutureWarning)
from pyprnt import prnt
from haversine import haversine
from folium.plugins import MarkerCluster
from folium import plugins

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [2]:
best_food_df = pd.read_excel("맛집현황.xlsx")
good_food_df = pd.read_excel('모범음식점현황.xlsx')

In [3]:
best_food_df.rename(columns ={'음식점명' : '식당', 'Unnamed: 4' : '최후의음식', 'WGS84위도' : '위도',  'WGS84경도' : '경도'},inplace=True)
good_food_df.rename(columns ={'업소명' : '식당', '최후의 음식' : '최후의음식', 'WGS84위도' : '위도',  'WGS84경도' : '경도'},inplace=True)
best_food_df = best_food_df[['시군명','소재지지번주소','식당','최후의음식','위도','경도']]
good_food_df = good_food_df[['시군명','소재지지번주소','식당','최후의음식','위도','경도']]
final_food_df = pd.concat([best_food_df,good_food_df])
final_food_df.drop_duplicates(inplace=True)

In [4]:
final_food_df_en = pd.read_csv("final_food_en.csv")
final_food_df['식당_영문명'] = final_food_df_en['식당_영문명']
final_food_df=  final_food_df[['시군명','소재지지번주소','식당','식당_영문명','최후의음식','위도','경도']]

In [23]:
final_food_df[final_food_df['최후의음식']=='짬뽕']

,시군명,소재지지번주소,식당,식당_영문명,최후의음식,위도,경도
3,고양시,경기도 고양시 일산서구 대화동 2101번지,정통중화요리 남궁,igyeongsughalmeonieumsigjeom,짬뽕,37.682042,126.753550
24,고양시,경기도 고양시 일산동구 정발산동 1262-1번지,뿅의 전설,ppyong-ui jeonseol,짬뽕,37.674687,126.783488
586,남양주시,경기도 남양주시 와부읍 팔당리 428번지,강릉짬뽕순두부,gangleungjjamppongsundubu,짬뽕,37.550321,127.238387
925,성남시,경기도 성남시 수정구 태평동 3564번지,바다해물짬뽕,badahaemuljjamppong,짬뽕,37.443914,127.134316
988,성남시,경기도 성남시 수정구 금토동 272-33번지,홍루원,hongluwon,짬뽕,37.409503,127.097716


In [18]:
x = [37.400253, 127.104999]

label = '짬뽕'

In [19]:
# label과 맞는 음식점 고르기
def match_food(df,label):  
    df = df[df['최후의음식']==label]

    return df

# label의 맞는 음식점에서 가까운 10개 고르기

def match_top10(x,df):
    dist = []
    match_food_df = match_food(df,label)
    for i in range(len(match_food_df)):
        dist.append(haversine(tuple(x),tuple(match_food_df[['위도','경도']].iloc[i])))

    match_food_df['dist'] = dist
    return  match_food_df.sort_values('dist',ascending=True).head(10)
   

In [20]:
match_top = match_top10(x,final_food_df)
match_food = match_food(final_food_df,label)
# 중복되는 행 제거
for i in match_top.index:
    match_food.drop(i, axis='index', inplace=True)

C:\Users\leejy\Anaconda3_\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\leejy\Anaconda3_\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
## 인덱스 번호가 다를 경우 drop시키는 것
# for i in match_top.index:
#     name = match_top.loc[i]['식당']
#     loc = (match_top.loc[i]['위도'], match_top.loc[i]['경도']) 
#     index = match_food[match_food['식당'] == name].index[0]
    
#     if (match_food.loc[index]['위도'] == loc[0]) & (match_food.loc[index]['경도'] == loc[1]):
#         match_food.drop(index, axis = 'index', inplace=True)

In [16]:
star_result = []
score_url= 'https://www.google.com/maps'
def get_star(target_res):
    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')
    driver.get(score_url)
    

    for i in target_res.index:
        driver.find_element_by_id('searchboxinput').clear()
        keyword = target_res.loc[i]['소재지지번주소'] +' ' + target_res.loc[i]['식당']
        driver.find_element_by_id('searchboxinput').send_keys(keyword)
        xpath = "//*[@id='searchbox-searchbutton']"
        driver.find_element_by_xpath(xpath).click()
       # driver.find_element_by_id('searchbox-searchbutton').click()
        time.sleep(5)

        star = driver.find_element_by_class_name('section-star-display')
        star_result.append(star)
        
        #print('{} \n ★ {}점'.format(final_food_df.iloc[i]['식당'], star_text))
    driver.close()
    driver.quit()
    return star_result

In [13]:
match_top['소재지지번주소'].iloc[1][0:-5]

'경기도 성남시 분당구 삼평동 '

In [17]:
match_top['star'] = get_star(match_top)

WebDriverException: Message: unknown error: unknown sessionId
  (Session info: chrome=84.0.4147.125)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Windows NT 10.0.18363 x86_64)


In [101]:
# star_result = {}
# score_url= 'https://www.google.com/maps'
# def get_star(target_res):
#     driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 
#     driver.get(score_url)
    

#     for i in target_res.index:
#         driver.find_element_by_id('searchboxinput').clear()
#         keyword = final_food_df.iloc[i]['소재지지번주소'] +' ' + final_food_df.iloc[i]['식당']
#         driver.find_element_by_id('searchboxinput').send_keys(keyword)
#         search = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
#         search.click()
#         time.sleep(5)

#         star = driver.find_element_by_class_name('section-star-display')
#         star_text = star.text
#         star_result[i] = star_text
#         print('{} \n ★ {}점'.format(final_food_df.iloc[i]['식당'], star_text))
#     driver.close()
#     driver.quit()

In [21]:
match_top

,시군명,소재지지번주소,식당,식당_영문명,최후의음식,위도,경도,dist
988,성남시,경기도 성남시 수정구 금토동 272-33번지,홍루원,hongluwon,짬뽕,37.409503,127.097716,1.213157
925,성남시,경기도 성남시 수정구 태평동 3564번지,바다해물짬뽕,badahaemuljjamppong,짬뽕,37.443914,127.134316,5.502020
586,남양주시,경기도 남양주시 와부읍 팔당리 428번지,강릉짬뽕순두부,gangleungjjamppongsundubu,짬뽕,37.550321,127.238387,20.420754
24,고양시,경기도 고양시 일산동구 정발산동 1262-1번지,뿅의 전설,ppyong-ui jeonseol,짬뽕,37.674687,126.783488,41.651388
3,고양시,경기도 고양시 일산서구 대화동 2101번지,정통중화요리 남궁,igyeongsughalmeonieumsigjeom,짬뽕,37.682042,126.753550,44.067686


In [14]:
match_top['소재지지번주소'].apply(lambda x : ' '.join(x.split()[1:-1]))

1033    성남시 분당구 삼평동
912     성남시 분당구 삼평동
1018    성남시 분당구 판교동
1101    성남시 분당구 이매동
1085    성남시 분당구 서현동
989     성남시 수정구 시흥동
1080    성남시 분당구 야탑동
941     성남시 중원구 여수동
1094    성남시 분당구 야탑동
1107    성남시 분당구 궁내동
Name: 소재지지번주소, dtype: object

In [24]:
# 지도 시각화
geo_path = 'Gyeonggi_municipalities-geo.json'
#경기도 행정구역만 정리된 json파일 불러오기
geo_str = json.load(open(geo_path, encoding='utf-8'))

geo_str['features'][0]

map_1 = folium.Map(
    location=[37.400253, 127.104999],
    zoom_start=13
)
# marker_cluster 만들어주는 부분
mcg = folium.plugins.MarkerCluster(control=False) 
map_1.add_child(mcg)

# subgroup 만드는 부분

g1 = plugins.FeatureGroupSubGroup(mcg,'Near 10 restaurant')
map_1.add_child(g1)
g2 = plugins.FeatureGroupSubGroup(mcg,'Other restaurant')
map_1.add_child(g2)

# icon_img 넣어주는 부분
icon_img = folium.features.CustomIcon('icon.png', icon_size=(50, 50))
marker_icon = folium.Marker([37.400253, 127.104999], 
                        tooltip='Here!',
                        icon=icon_img)

marker_icon.add_to(map_1)






with open(geo_path,mode='rt',encoding='utf-8') as f:
    geo = json.loads(f.read())
    f.close()


str_x = str(x)

for i in range(len(match_top)):
    # 행 우선 접근 방식으로 값 추출하기
    temp = []
    name_top = match_top.iloc[i]['식당_영문명']
    name_top_star = match_top.iloc[i]['식당_영문명'] #+ ' ★ ' + match_top.iloc[i]['star']
    lat_top = match_top.iloc[i]['위도']
    lng_top = match_top.iloc[i]['경도']
    icon = folium.Icon(icon='star',color='red')
    # 추출한 정보를 지도에 표시
    
    temp.append(lat_top)
    temp.append(lng_top)
    temp = str(temp)
    
    start = str_x[1:-1]
    end = temp[1:-1]
    url ="https://www.google.com/maps/dir/"+ start +  "/" + end + "/@37.3956279,126.8705842,11z/data=!3m1!4b1?hl=en"
    tooltip =  name_top_star
#   folium.Popup('<a href=" [URL GOES HERE] "target="_blank"> [text for link goes here]' </a>')
    popup = folium.Popup('<a href=\"'+url+ '"> How to go to '+ name_top +'  </a>', max_width=200)
    
    marker_top = folium.Marker([lat_top,lng_top],popup=popup,tooltip= tooltip,icon=icon)

    marker_top.add_to(g1)
    
for i in range(len(match_food)):
    
    name = match_food.iloc[i]['식당']
    lat = match_food.iloc[i]['위도']
    lng = match_food.iloc[i]['경도']

    # 추출한 정보를 지도에 표시
    popup_other = folium.Popup(name, max_width=200)
    marker = folium.Marker([lat,lng], popup=popup_other)
    marker.add_to(g2)
    
folium.LayerControl(collapsed=False).add_to(map_1)
map_1.save('Foodmap.html')
map_1

#한글 깨지는 문제는 html 파일로 저장하면 문제 해결_

In [ ]:
plugins.